# 학습 데이터 저장

In [ ]:
import urllib.request as REQ
from bs4 import BeautifulSoup
from urllib import parse
import pandas as pd

u = "https://www.klac.or.kr/legalinfo/counselView.do?pageIndex=1&folderId=006&caseId=case-"
numList = ['004', '005', '006', '007', '008', '011', '012'] #각각 손해배상, 민사일반, 물권, 채권, 친족, 계약, 상속
sentList1 = [] #제목 리스트(질문이 짧게 요약된 형태, 제목도 bert 학습 시 사용)
sentList2 = [] #질문 리스트
labelList = [] #레이블 리스트 손해배상이면 0, 민사일반이면 1 … 상속이면 6
df = pd.DataFrame(columns=['title', 'question', 'answer', 'label'])
for n in range(0, len(numList)):
  i = 1
  while True:
    num = str(i)
    for k in range(5-len(num)):
      num = '0' + num
    url = u + numList[n] + '-' + num
    print(url)
    try:
      html = REQ.urlopen(url).read()
      soup = BeautifulSoup(html, "html.parser")
      try:
        wList1 = soup.select('dd')
        if len(wList1[4].text) == 0:
          break
        sentList1.append(wList1[4].text) #제목 추가
        sentList2.append(wList1[5].text) #질문 추가
        labelList.append(n)
      except:
        print('오류')
    except:
      print('오류')
    i += 1
qDic = {}
qDic['title'] = sentList1
qDic['question'] = sentList2
qDic['label'] = labelList
df = pd.DataFrame(qDic, columns = ['title', 'question', 'label'])
df.index = range(len(df))
df.to_pickle("/content/drive/MyDrive/파일위치/q_7400.pkl")


# kobert 학습

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
#kobert
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model
#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
import pandas as pd
train_data = pd.read_pickle("/content/drive/MyDrive/파일위치/q_7400.pkl")
#GPU 사용
device = torch.device("cuda:0")
#BERT 모델, Vocabulary 불러오기
bertmodel, vocab = get_pytorch_kobert_model()
dList = []
for t, q, label in zip(train_data['title'], train_data['question'], train_data['label'])  :
    data = []
    data.append(q)
    data.append(str(label))
    dList.append(data)
    data = []
    data.append(t)
    data.append(str(label))
    dList.append(data)

#train & test 데이터로 나누기
from sklearn.model_selection import train_test_split
data_list, dataset_val = train_test_split(dList, test_size=0.1, random_state=0)
dataset_train, dataset_test = train_test_split(data_list, test_size=0.2, random_state=0)

class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]
    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))
    def __len__(self):
        return (len(self.labels))

#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

data_train = BERTDataset(dataset_train[:10], 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test[:5], 0, 1, tok, max_len, True, False)

train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=2, num_workers=0)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=2, num_workers=0)

class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=7,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate

        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
        print( self.classifier)
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        print('token_ids', token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        print('attention_mask', attention_mask)
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)

        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        print('valid_length' , valid_length)
        print('segment_ids' , segment_ids)
        return self.classifier(out)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#BERT 모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

#BERT 모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    print('??', max_vals, max_indices)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

train_dataloader

checkpoint_epoch = -1
checkpoint = torch.load('./checkpoint-4.pt')
model.load_state_dict(checkpoint["model_state_dict"])
optimizer.load_state_dict(checkpoint["optimizer_state_dict"])
checkpoint_epoch = checkpoint["epoch"]
checkpoint_description = checkpoint["description"]
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
model.eval()
for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))
